<a href="https://colab.research.google.com/github/Ixiondbz/CSE499/blob/main/RCNN_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!pip install tensorflow==2.7.0
!pip install -U plotly

import json
from pathlib import Path
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm import tqdm
import pandas as pd
import random
import cv2

import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import math
import cv2
import copy
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import os

from sklearn.metrics import average_precision_score

from keras import backend as K
from tensorflow.keras.optimizers import SGD, RMSprop
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
from keras.utils.layer_utils import get_source_inputs
#from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from tensorflow.keras.metrics import categorical_crossentropy

from keras.models import Model
from keras.utils import generic_utils
#tf.keras.layers.InputSpec
from tensorflow.keras.layers import Layer, InputSpec
from keras import initializers, regularizers

In [25]:
from google.colab import drive
drive.mount('/drive')

folder = "MyDrive/MIDOG_Challenge" #@param {type:"string"}
midog_folder = Path("/drive") / Path(folder)

print(list(midog_folder.glob("*.*")))

Mounted at /drive
[PosixPath('/drive/MyDrive/MIDOG_Challenge/MIDOG.sqlite'), PosixPath('/drive/MyDrive/MIDOG_Challenge/MIDOG.json')]


In [26]:
!apt-get install python3-openslide
from openslide import open_slide

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-openslide is already the newest version (1.1.1-2ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [27]:
image_folder = midog_folder / "images"

hamamatsu_rx_ids = list(range(0, 51))
hamamatsu_360_ids = list(range(51, 101))
aperio_ids = list(range(101, 151))
leica_ids = list(range(151, 201))


annotation_file = midog_folder / "MIDOG.json"
rows = []
with open(annotation_file) as f:
    data = json.load(f)

    categories = {1: 'mitotic figure', 2: 'hard negative'}

    for row in data["images"]:
        file_name = row["file_name"]
        image_id = row["id"]
        width = row["width"]
        height = row["height"]

        scanner  = "Hamamatsu XR"
        if image_id in hamamatsu_360_ids:
            scanner  = "Hamamatsu S360"
        if image_id in aperio_ids:
            scanner  = "Aperio CS"
        if image_id in leica_ids:
            scanner  = "Leica GT450"
         
        for annotation in [anno for anno in data['annotations'] if anno["image_id"] == image_id]:
            box = annotation["bbox"]
            cat = categories[annotation["category_id"]]

            rows.append([file_name, image_id, width, height, box, cat, scanner])

df = pd.DataFrame(rows, columns=["file_name", "image_id", "width", "height", "box", "cat", "scanner"])

In [28]:
train = df

In [29]:
train.head()

,file_name,image_id,width,height,box,cat,scanner
0,001.tiff,1,7215,5412,"[4336, 346, 4386, 396]",hard negative,Hamamatsu XR
1,001.tiff,1,7215,5412,"[756, 872, 806, 922]",hard negative,Hamamatsu XR
2,001.tiff,1,7215,5412,"[270, 4044, 320, 4094]",hard negative,Hamamatsu XR
3,001.tiff,1,7215,5412,"[6672.5, 706.5, 6722.5, 756.5]",hard negative,Hamamatsu XR
4,002.tiff,2,7215,5412,"[1872, 319, 1922, 369]",hard negative,Hamamatsu XR


In [30]:
train[train['file_name']=='001.tiff']

,file_name,image_id,width,height,box,cat,scanner
0,001.tiff,1,7215,5412,"[4336, 346, 4386, 396]",hard negative,Hamamatsu XR
1,001.tiff,1,7215,5412,"[756, 872, 806, 922]",hard negative,Hamamatsu XR
2,001.tiff,1,7215,5412,"[270, 4044, 320, 4094]",hard negative,Hamamatsu XR
3,001.tiff,1,7215,5412,"[6672.5, 706.5, 6722.5, 756.5]",hard negative,Hamamatsu XR


In [31]:
train['cat'].unique()

array(['hard negative', 'mitotic figure'], dtype=object)

In [32]:
train.dtypes

file_name    object
image_id      int64
width         int64
height        int64
box          object
cat          object
scanner      object
dtype: object

In [33]:
train['box']

0               [4336, 346, 4386, 396]
1                 [756, 872, 806, 922]
2               [270, 4044, 320, 4094]
3       [6672.5, 706.5, 6722.5, 756.5]
4               [1872, 319, 1922, 369]
                     ...              
4430            [569, 4276, 619, 4326]
4431          [3730, 4538, 3780, 4588]
4432          [4318, 4138, 4368, 4188]
4433          [5643, 1318, 5693, 1368]
4434      [2681.5, 2846, 2731.5, 2896]
Name: box, Length: 4435, dtype: object

In [34]:
box_dataframe = train.copy(deep=True)

In [35]:
#df.drop(df[df['Age'] < 25].index, inplace = True)
#box_dataframe.drop(box_dataframe[box_dataframe['cat']=="hard negative"].index,inplace=True)

In [38]:
box_dataframe.shape

(1721, 7)

In [ ]:
box_dataframe

,image_id,box
0,1,"[4336, 346, 4386, 396]"
1,1,"[756, 872, 806, 922]"
2,1,"[270, 4044, 320, 4094]"
3,1,"[6672.5, 706.5, 6722.5, 756.5]"
4,2,"[1872, 319, 1922, 369]"
...,...,...
4430,150,"[569, 4276, 619, 4326]"
4431,150,"[3730, 4538, 3780, 4588]"
4432,150,"[4318, 4138, 4368, 4188]"
4433,150,"[5643, 1318, 5693, 1368]"


In [ ]:
size = box_dataframe.shape[0]

In [ ]:
List = box_dataframe[box_dataframe['image_id']==1]['box']
List 

0            [4336, 346, 4386, 396]
1              [756, 872, 806, 922]
2            [270, 4044, 320, 4094]
3    [6672.5, 706.5, 6722.5, 756.5]
Name: box, dtype: object

In [ ]:
%cd '/content/drive/MyDrive/CSE499 Project/annotations'

/content/drive/MyDrive/CSE499 Project/annotations


In [ ]:
FileName = str(1)+".txt"
print(FileName)

1.txt


In [ ]:
for i in range(150):
  FileName = str(i+1)+".txt"
  File = open(FileName, "a")
  ListOfLists = box_dataframe[box_dataframe['image_id']==(i+1)]['box']
  
  for List in ListOfLists:
    x_center = List[0] + (List[2] - List[0]) / 2
    y_center = List[1] + (List[3] - List[1]) / 2
    box_wt   = List[2] - List[0]
    box_ht   = List[3] - List[1]
    File.write(str(0)+" "+str(x_center)+" "+str(y_center)+" "+str(box_wt)+" "+str(box_ht))
    File.write('\n')
  File.close()